In [ ]:
import pandas as pd

# Đọc file Excel, bỏ qua hàng chứa tiêu đề "Route"
file_path = "/content/Book1.xlsx"
df = pd.read_excel(file_path, header=1)

df.columns = ["From", "To", "From_id", "To_id", "Leadtime", "Cost"]
# In ra dữ liệu trong sheet đã đọc
print(df.head())

       From       To From_id To_id  Leadtime  Cost
0  Hưng Yên   Hà Nội      P1    D1       0.3   0.3
1  Hưng Yên  Đà Nẵng      P1    D2       3.0   3.0
2   Thủ Đức   Hà Nội      P2    D1       7.0   8.0
3   Thủ Đức  Đà Nẵng      P2    D2       6.0   6.0
4   Thủ Đức   TP.HCM      P2    D3       0.3   0.3


In [ ]:
import heapq

def dijkstra(graph, start, end):
    heap = [(0, start)]
    visited = set()
    distances = {start: 0}
    predecessors = {}  # Lưu trữ các đỉnh trước đỉnh hiện tại trên đường đi ngắn nhất

    while heap:
        (cost, node) = heapq.heappop(heap)

        if node == end:
            path = []
            while node is not None:
                path.append(node)
                node = predecessors.get(node)
            path.reverse()
            return [start, path, distances[end]]

        if node not in visited:
            visited.add(node)

            for neighbor, weight in graph[node].items():
                if neighbor not in visited:
                    new_cost = round(cost + weight,4)
                    if new_cost < distances.get(neighbor, float('inf')):
                        distances[neighbor] = new_cost
                        predecessors[neighbor] = node
                        heapq.heappush(heap, (new_cost, neighbor))

    return None  # Trả về None nếu không có đường đi từ start đến end

# Bảng giá vận chuyển
shipping_costs = {}
for index, row in df.iterrows():
    from_code = row['From_id']
    to_code = row['To_id']
    cost = row['Cost']
    shipping_costs[(from_code, to_code)] = cost

# Tạo đồ thị mới với tất cả đường có thể đi
graph = {}
for (source, dest), cost in shipping_costs.items():
    if source not in graph:
        graph[source] = {}
    if dest not in graph:
        graph[dest] = {}
    graph[source][dest] = cost

# Tạo đường đi ngắn nhất từ tất cả nhà máy và kho đến khách hàng
def cheapest_shipping_costs(graph, start_nodes, end_node):
    cheapest_costs = []
    for start in start_nodes:
      distances = dijkstra(graph, start, end_node)
      cheapest_costs.append(distances)
    return cheapest_costs

In [ ]:
def min_cost(arr): #Tìm chi phí nhỏ nhất trong tất cả đường đi
  min = arr[0]
  for i in arr[1:]:
    if i != None:
      if min==None:
        min=i
      elif i[2]<min[2]:
        min=i
  return min

In [ ]:
unique_fromid = df['From_id'].unique()
unique_fromid

array(['P1', 'P2', 'D1', 'D3', 'D2'], dtype=object)

In [ ]:
import copy
def modify_graph_d(graph,warehouse1,warehouse2):
  modified_data = copy.deepcopy(graph)
  for key, value in modified_data.items():
    if key.startswith('D'):
        for inner_key in list(value.keys()):
            if inner_key.startswith('C') and key not in [warehouse1,warehouse2]:
                if inner_key in modified_data[key]:
                #print(key,inner_key)
                  del modified_data[key][inner_key]
  return modified_data



In [ ]:
def solution(customer, unique_fromid, order, stock, graph):
  cost = 0
  result = []

  #Nếu tất cả lượng tồn kho không đủ thì trả về -1
  total_stock=sum(stock.values())
  if total_stock < order:
    return -1

  #Tìm đường đi ngắn nhất từ các nhà máy, kho đến khách hàng.
  sorted_paths = cheapest_shipping_costs(graph, unique_fromid, customer)

  #Nếu kho có chi phí thấp nhất đủ hàng thì giao trực tiếp
  min = min_cost(sorted_paths) #
  if min==None:     #Nếu không có đường nào vận chuyển đến khách hàng.
    return None
  elif stock[min[0]]>=order:
    return [min[1],order*min[2]]



  temp = []
  for i in range(len(sorted_paths)-1):
    for j in range(i+1,len(sorted_paths)):
      #Nếu bắt cặp rỗng hoặc bắt cặp với nhà máy thì bỏ qua
      if sorted_paths[i] == None or sorted_paths[j] == None:
        continue
      elif sorted_paths[i][0].startswith('P') or sorted_paths[j][0].startswith('P'):
        continue
      else:
        modify_graph = modify_graph_d(graph,sorted_paths[i][0],sorted_paths[j][0]) #Xoá hết tất cả đường đi từ kho đến khách hàng, chỉ giữ lại cặp đã chọn
        temp = cheapest_shipping_costs(modify_graph, unique_fromid, customer) #
        result_temp = []
        total_cost = 0
        temp_order = order

        #Duyệt qua từng nhà máy, kho vận chuyển có chi phí đến khách hàng thấp nhất
        #Xem cái nào chi phí thấp nhất thì ưu tiên lấy hàng ở đó trước
        while temp_order>0 and len(temp)>0:
          min = min_cost(temp)      #Nơi có phí vận chuyển thấp nhất
          if min == None:     #Nếu nơi đó không có đường đi thì tiếp tục duyệt qua nơi khác
            temp.remove(min)
            continue
          elif stock[min[0]] >= temp_order:  #Nếu chỗ đó đủ hàng thì dừng lại và lưu kết quả
            result_temp.append(min[1:])
            total_cost += min[2]*stock[min[0]]
            temp_order = 0
            break
          elif stock[min[0]]<=0:    #Nếu chỗ đó không có hàng thì bỏ qua
            temp.remove(min)
            continue
          else:                         #Nếu nơi đó vẫn chưa đủ hàng thì lấy hết hàng nơi đó và giảm bớt lượng order
            result_temp.append(min[1:])
            total_cost += min[2]*stock[min[0]]
            temp_order -= stock[min[0]]
            temp.remove(min)            #Bỏ ra khỏi danh sách duyệt
        if temp_order==0:
          result.append([result_temp,total_cost])
  min = result[0]
  for i in result[1:]:
    if i[1] < min[1]:
      min = i
  #print(result)
  return min


In [ ]:
#Khởi tạo stock để tính thử
stock = {}
for i in unique_fromid:
  stock[i] = 2


#Thử với khách hàng C1 và order 10 món hàng
print(solution("C1",unique_fromid,10,stock,graph))

[[[['D1', 'C1'], 1.2], [['P1', 'D1', 'C1'], 1.5], [['D2', 'C1'], 3.0], [['P2', 'D2', 'C1'], 9.0], [['D3', 'D2', 'C1'], 9.0]], 47.4]
